# Leer

In [ ]:
# Setup
!pip install -q langchain langchain-community  wikipedia chromadb
import os
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!apt-get install -y poppler-utils
!apt-get install tesseract-ocr
!pip install "unstructured[pdf]" "pytesseract" "pdf2image" "langchain" "pymupdf" "python-docx"
!pip install -qU langchain-community unstructured pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (298 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed

In [ ]:
!pip install unstructured pdfminer.six python-docx pi_heif unstructured_inference

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("./ec2-ug.pdf")
ec2documents = loader.load()
print(f"{len(ec2documents)} documento(s) cargado(s).")


1 documento(s) cargado(s).


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("./rds-ug.pdf")
rdsdocuments = loader.load()
print(f"{len(rdsdocuments)} documento(s) cargado(s).")


1 documento(s) cargado(s).


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("./s3-userguide.pdf")
s3documents = loader.load()
print(f"{len(s3documents)} documento(s) cargado(s).")


1 documento(s) cargado(s).


# chunkeo

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
print("\n--- Using RecursiveCharacterTextSplitter ---")
rec_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
ec2_splits = rec_splitter.split_documents(ec2documents)
rds_splits = rec_splitter.split_documents(rdsdocuments)
s3_splits = rec_splitter.split_documents(s3documents)
print(f"ec2 Split into {len(ec2_splits)} chunks")
print(f"rds Split into {len(rds_splits)} chunks")
print(f"s3 Split into {len(s3_splits)} chunks")



--- Using RecursiveCharacterTextSplitter ---
rds Split into 24812 chunks


# Conversion a embbeding

In [ ]:
!pip install langchain_chroma langchain_huggingface langchain_pymupdf4llm

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
from langchain_text_splitters import CharacterTextSplitter

embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
# Función auxiliar para dividir en lotes
def batch_documents(docs, batch_size):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]

In [ ]:
#!unzip ec2_chroma_vectorstore.zip -d ec2_chroma_vectorstore

Archive:  ec2_chroma_vectorstore.zip
   creating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/
  inflating: ec2_chroma_vectorstore/chroma.sqlite3  
  inflating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/header.bin  
  inflating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/data_level0.bin  
  inflating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/link_lists.bin  
  inflating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/length.bin  
  inflating: ec2_chroma_vectorstore/277ea1d1-8b75-4cd4-a790-f50558cd4b5d/index_metadata.pickle  


In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma


ec2vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./ec2_chroma_vectorstore"
)

# Agregar por lotes
for batch in batch_documents(ec2_splits, batch_size=5000):
    ec2vector_store.add_documents(documents=batch)



In [ ]:
rdsvector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./rds_chroma_vectorstore"
)

for batch in batch_documents(rds_splits, batch_size=5000):
  rdsvector_store.add_documents(documents=batch)



In [ ]:
s3vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./s3_chroma_vectorstore"
)

for batch in batch_documents(s3_splits, batch_size=5000):
  s3vector_store.add_documents(documents=batch)

In [ ]:
import shutil
from google.colab import files

# Comprimir la carpeta (reemplaza 'mi_carpeta' con el nombre de tu carpeta)
shutil.make_archive('ec2_chroma_vectorstore', 'zip', 'ec2_chroma_vectorstore')

# Descargar el archivo ZIP
files.download('ec2_chroma_vectorstore.zip')

# Comprimir la carpeta (reemplaza 'mi_carpeta' con el nombre de tu carpeta)
shutil.make_archive('rds_chroma_vectorstore', 'zip', 'rds_chroma_vectorstore')

# Descargar el archivo ZIP
files.download('rds_chroma_vectorstore.zip')

# Comprimir la carpeta (reemplaza 'mi_carpeta' con el nombre de tu carpeta)
shutil.make_archive('s3_chroma_vectorstore', 'zip', 's3_chroma_vectorstore')

# Descargar el archivo ZIP
files.download('s3_chroma_vectorstore.zip')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generacion de respuestas

In [ ]:
#Descarga de OLlama
!curl -fsSL https://ollama.com/install.sh | sh  # Solo una vez

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#Lanzar servidor de OLlama en local
!ollama serve > /dev/null 2>&1 &
!sleep 10

In [ ]:
#Descarga del modelo y librerías
!ollama pull phi3.5:latest
!pip install -U langchain-ollama

In [ ]:
# Carga del modelo
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="phi3.5:latest",
    temperature=0.3,
    num_gpu=40,
    system=""
)

In [ ]:
# Configuración del pipeline RAG
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


# Plantilla personalizada solo es una plantilla para hacer el promt
prompt = ChatPromptTemplate.from_template("""
Answer in spanish briefly

{context}

Question: {input}
Answer:""")

#este es el que saca los vectores por similaridad de la base de chunks
retriever = ec2vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Traer top 3 chunks
)

# Crear cadena RAG
combine_docs_chain = create_stuff_documents_chain(llm, prompt) #esto configura el llm para generar usando la plantilla
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain) #esto agrega los 3 chunks encontrados al promt


# Paso 1: Crear una cadena para traducir preguntas al inglés
translate_prompt = PromptTemplate.from_template(
    "Traduce la siguiente pregunta del español al inglés sin añadir comentarios:\n{pregunta}"
)

translate_chain = LLMChain(llm=llm, prompt=translate_prompt)

pregunta_usuario = "¿Cómo me puedo conectar a mi instancia?"
pregunta_traducida = translate_chain.invoke({"pregunta": pregunta_usuario})["text"]

# Ejecutar consulta
result = retrieval_chain.invoke({
    "input": pregunta_traducida
})


print("Respuesta:", result["answer"])
print("\nChunks recuperados:")
for doc in result["context"]:
    print(f"\n- {doc.page_content[:1000]}...")


Respuesta: 1. Vaya a la página Instancias en AWS y seleccione su instancia marcada por una casilla de verificación. Luego, haga clic en "Conectar".
2. Use el comando `aws ec2-instance-connect ssh --instance-id i-1234567890example --connection-type eice`.
3. Para usar EC2 Instance Connect con su clave privada, especifique la instancia ID y ruta al archivo de clave sin `file://`, por ejemplo: `/path/to/key` como argumento del comando. Además, asegúrese de que tiene las direcciones IP públicas o privadas necesarias para conectarse usando esta herramienta.
4. Asegúrese de tener un par compatible (RSA o ED25519) y una longitud válida (2048 o 4096 bits). Para más detalles, consulte cómo crear uno con una utilidad externa.

Chunks recuperados:

- Siga estos pasos para conectarse a la instancia:

1.

Inicie la conexión:

Si se encuentra en la página de detalles de la instancia de la consola de Amazon EC2, pulse el

botón Conectar (arriba a la derecha).

Si ha salido de la página, seleccione In